### 0. 시작해봅시다

In [1]:
#항상 이 두 줄은 치고 시작합시당
import numpy as np
import pandas as pd

In [2]:
#파일 불러옵시당
loans=pd.read_csv("loan_data.csv")
loans.head()

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


In [3]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   credit.policy      9578 non-null   int64  
 1   purpose            9578 non-null   object 
 2   int.rate           9578 non-null   float64
 3   installment        9578 non-null   float64
 4   log.annual.inc     9578 non-null   float64
 5   dti                9578 non-null   float64
 6   fico               9578 non-null   int64  
 7   days.with.cr.line  9578 non-null   float64
 8   revol.bal          9578 non-null   int64  
 9   revol.util         9578 non-null   float64
 10  inq.last.6mths     9578 non-null   int64  
 11  delinq.2yrs        9578 non-null   int64  
 12  pub.rec            9578 non-null   int64  
 13  not.fully.paid     9578 non-null   int64  
dtypes: float64(6), int64(7), object(1)
memory usage: 1.0+ MB


In [ ]:
#간단한 변수 설명
'''
For this project we will be exploring publicly available data from LendingClub.com. 
Lending Club connects people who need money (borrowers) with people who have money (investors). 
Hopefully, as an investor you would want to invest in people who showed a profile of having a high probability of paying you back. 
We will try to create a model that will help predict this.
We will use lending data from 2007-2010 and try to classify and predict whether or not the borrower paid back their loan in full.
'''
#그래서! 타겟 변수 (target variable)은 not.fully.paid랍니다

In [4]:
#전처리 살짝만 할게용
'''
지금 범주형 데이터가 있는데요! 저기 Dtype에서 'object' 보이시죠?
이들 항목이 각각 하나의 컬럼으로 들어가도록 더미 변수 생성해줄게요.
'''

loans['purpose'].unique()

array(['debt_consolidation', 'credit_card', 'all_other',
       'home_improvement', 'small_business', 'major_purchase',
       'educational'], dtype=object)

In [5]:
cat_feats=['purpose']

#컬럼이 늘어났을거예요! shape를 살펴봅시다 (R에서 dim 생각해보시면..!)
final_data=pd.get_dummies(loans,columns=cat_feats, drop_first=True)
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   credit.policy               9578 non-null   int64  
 1   int.rate                    9578 non-null   float64
 2   installment                 9578 non-null   float64
 3   log.annual.inc              9578 non-null   float64
 4   dti                         9578 non-null   float64
 5   fico                        9578 non-null   int64  
 6   days.with.cr.line           9578 non-null   float64
 7   revol.bal                   9578 non-null   int64  
 8   revol.util                  9578 non-null   float64
 9   inq.last.6mths              9578 non-null   int64  
 10  delinq.2yrs                 9578 non-null   int64  
 11  pub.rec                     9578 non-null   int64  
 12  not.fully.paid              9578 non-null   int64  
 13  purpose_credit_card         9578 

In [6]:
final_data.shape

(9578, 19)

### 1. train-test split

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X = final_data.drop('not.fully.paid',axis=1)
y = final_data['not.fully.paid']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=2021)

In [9]:
#X_train, X_test shape 살펴보아요
print(X_train.shape)
print(X_test.shape)

(6704, 18)
(2874, 18)


### 2. 모델링

In [10]:
#RandomForestClassifier를 분류기로 사용해보려고 합니다. 자세한 설명은 다음주에 할게요~
#sklearn의 RandomForestClassifier는 엄청~나게 많은 파라미터를 지닙니다. 공식문서 참고!
from sklearn.ensemble import RandomForestClassifier

In [14]:
#1. RandomizedSearchCV를 통해 최적의 파라미터를 찾아봅시다.
#이 파라미터 각각이 무슨 의미인지는 지금은 몰라도 돼요!

from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [17]:
#5-fold cv해보고, 총 5번의 iteration 동안 돌려볼게요.
randomized_rf=RandomForestClassifier()
cv_randomized_rf = RandomizedSearchCV(estimator = randomized_rf, 
                                      param_distributions = random_grid, 
                                      n_iter = 5, cv = 5, 
                                      verbose=2, random_state=2021)
cv_randomized_rf.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True, total=   4.8s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.7s remaining:    0.0s


[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True, total=   4.7s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True, total=   4.8s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True, total=   4.7s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=600, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True, total=   4.7s
[CV] n_estimators=400, min_samples_split=5, min_samples_leaf=4, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=400, min_

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  3.0min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=5,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000]},
                   random_state=2021, verbose=2)

In [18]:
#최적의 파라미터 찾기!
cv_randomized_rf.best_params_

{'n_estimators': 400,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': False}

In [19]:
#2. GridSearchCV를 통해 최적의 파라미터를 찾아봅시다.
'''
RandomizedSearchCV가 특정 범위 내에서 최적의 파라미터를 찾는 것이라면 
GridSearchCV는 random search를 통해서 얻은 정보를 활용, 
더 세부적인 범위의 값을 직접적으로 지정하여 파라미터 탐색.

궁금한 점은 구글링을 통해 더 찾아보자구요~
''' 

#직접 해보기! 위의 RandomSearchCV 코드를 참고하여 GridSearchCV 코드를 작성해봅시다.
'''
주의! 각 파라미터에 대해 여러 개의 경우의 수를 부여할 경우 엄청엄청
오래 걸릴 거라는 거..우선은 코드만 작성해보고, 실행은 시간 많을 때 해봐용
'''

### 3. 성능 평가

In [25]:
from sklearn.metrics import classification_report, confusion_matrix

In [27]:
#1. randomized~
random_model=RandomForestClassifier(n_estimators=400, min_samples_split=5,
                                   min_samples_leaf=4, max_features='sqrt',
                                   max_depth=10, bootstrap=False)
random_model.fit(X_train,y_train)
random_predictions=random_model.predict(X_test)

In [28]:
print(classification_report(y_test,random_predictions))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      2401
           1       0.67      0.01      0.02       473

    accuracy                           0.84      2874
   macro avg       0.75      0.50      0.46      2874
weighted avg       0.81      0.84      0.76      2874



In [29]:
print(confusion_matrix(y_test,random_predictions))

[[2399    2]
 [ 469    4]]


In [ ]:
#2. grid~
#직접 해보기! 위 코드를 참고, GridSearchCV에 대한 성능을 조회하는 코드를 작성해봐요